# Room for the river: Gorssel 

In [1]:
import sys

print("User Current Version:-", sys.version)

User Current Version:- 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]


In [29]:
#imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from ema_workbench import (Model, CategoricalParameter,ScalarOutcome, IntegerParameter, RealParameter,
                           MultiprocessingEvaluator, ema_logging, Constant, Policy, Scenario,
                           perform_experiments, SequentialEvaluator,Constraint)
from problem_formulation import get_model_for_problem_formulation

from dike_model_function import DikeNetwork  # @UnresolvedImport

from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress,GenerationalBorg)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
from ema_workbench.analysis import plotting, plotting_util, parcoords, feature_scoring, prim
from ema_workbench import load_results 

from mpl_toolkits.mplot3d import Axes3D 

from sklearn import preprocessing

import time
import copy

import itertools

from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import os
import functools

import random

In [40]:
from scipy.spatial.distance import pdist, squareform

def evaluate_diversity_single(indices, distances, weight=0.5, distance='euclidean'):
    '''
    takes the outcomes and selected scenario set (decision variables), 
    returns a single 'diversity' value for the scenario set.
    outcomes : outcomes dictionary of the scenario ensemble
    decision vars : indices of the scenario set
    weight : weight given to the mean in the diversity metric. If 0, only minimum; if 1, only mean
    '''
    i, j = [e for e in zip(*itertools.combinations(indices, 2))]
    subset_distances = distances[i, j]
    minimum = np.min(subset_distances)
    mean = np.mean(subset_distances)
    diversity = (1-weight)*minimum + weight*mean
    
    return [diversity]


def find_maxdiverse_scenarios(distances, combinations):
    scores = []
    for indices in combinations:
        diversity = evaluate_diversity_single(indices, distances)
        scores.append((diversity, indices))

    return scores

def optimize(scenario, nfe, model, converge_metrics, epsilons):


    with MultiprocessingEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=epsilons,
                                     reference=scenario)
    return results, convergence


# Scenario Selection 

In [4]:
#Identify policies based on open exploration 

# policies = [Policy('policy 1', **{'0_RfR 0':1,
#                                   '0_RfR 1':1,
#                                   '0_RfR 2':1,
#                                   'A.1_DikeIncrease 0':5}),
#            Policy('policy 2', **{'4_RfR 0':1,
#                                   '4_RfR 1':1,
#                                   '4_RfR 2':1,
#                                   'A.5_DikeIncrease 0':5}),
#            Policy('policy 3', **{'1_RfR 0':1,
#                                   '2_RfR 1':1,
#                                   '3_RfR 2':1,
#                                   'A.3_DikeIncrease 0':5})]



In [5]:
#running the model through EMA workbench
ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3) 


In [6]:
with SequentialEvaluator(dike_model) as evaluator:
    results_MORDM = evaluator.perform_experiments(scenarios=10, policies=5) 
    #s for number of scenarios, p for number of experiments 
experiments_MORDM, outcomes_MORDM = results_MORDM 
#s for number of scenarios, p for number of experiments 

[MainProcess/INFO] performing 10 scenarios * 5 policies * 1 model(s) = 50 experiments
100%|██████████████████████████████████████████| 50/50 [01:03<00:00,  1.27s/it]
[MainProcess/INFO] experiments finished


In [7]:
#save the initial results to be sure the original can be reused without having to run the model again 
raw_outcomes    = outcomes_MORDM.copy()
raw_experiments = experiments_MORDM.copy()

In [8]:
df_experiments = pd.DataFrame(experiments_MORDM)

df_outcomes =pd.DataFrame()
for outcome in outcomes_MORDM.keys():
    df_outcomes[outcome] = outcomes_MORDM[outcome]

df_experiments
df_experiments.columns

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model'],
      dtype='object')

In [10]:
experiments = df_experiments.copy()
outcomes = df_outcomes.copy()

In [11]:
outcomes.keys()

Index(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs',
       'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage',
       'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths',
       'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs',
       'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage',
       'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
       'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs',
       'A.5_Expected Number of Deaths', 'RfR Total Costs',
       'Expected Evacuation Costs'],
      dtype='object')

In [12]:
max_evacuation_costs = 5000
max_expected_annual_damage = 1

In [13]:
outcomes[['A.4_Expected Annual Damage','A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
          'RfR Total Costs','Expected Evacuation Costs']]

outcomes_of_interest = outcomes[(outcomes['A.4_Expected Annual Damage']< max_expected_annual_damage) 
                                &( outcomes['Expected Evacuation Costs']<max_evacuation_costs)]

y = (outcomes['A.4_Expected Annual Damage']< max_expected_annual_damage)&( outcomes['Expected Evacuation Costs']<max_evacuation_costs)
#y

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11    False
12     True
13     True
14    False
15     True
16    False
17    False
18    False
19     True
20     True
21     True
22     True
23     True
24    False
25     True
26    False
27    False
28    False
29     True
30     True
31     True
32     True
33     True
34     True
35     True
36     True
37    False
38     True
39     True
40     True
41     True
42     True
43     True
44     True
45     True
46     True
47     True
48     True
49    False
dtype: bool

In [27]:
outcomes_of_interest.head()
#shape(outcomes_of_interest) 

,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Dike Investment Costs,A.2_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Dike Investment Costs,A.3_Expected Number of Deaths,A.4_Expected Annual Damage,A.4_Dike Investment Costs,A.4_Expected Number of Deaths,A.5_Expected Annual Damage,A.5_Dike Investment Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,0.0,1.842499e+08,0.0,0.0,1.128326e+08,0.0,0.0,1.433983e+08,0.0,0.0,2.922928e+07,0.0,1.371917e+07,1.708431e+08,0.004439,958900000.0,404.5171
1,0.0,1.842499e+08,0.0,0.0,1.128326e+08,0.0,0.0,1.433983e+08,0.0,0.0,2.922928e+07,0.0,0.000000e+00,1.708431e+08,0.000000,958900000.0,0.0000
2,0.0,1.842499e+08,0.0,0.0,1.128326e+08,0.0,0.0,1.433983e+08,0.0,0.0,2.922928e+07,0.0,0.000000e+00,1.708431e+08,0.000000,958900000.0,0.0000
3,0.0,1.842499e+08,0.0,0.0,1.128326e+08,0.0,0.0,1.433983e+08,0.0,0.0,2.922928e+07,0.0,0.000000e+00,1.708431e+08,0.000000,958900000.0,0.0000
4,0.0,1.842499e+08,0.0,0.0,1.128326e+08,0.0,0.0,1.433983e+08,0.0,0.0,2.922928e+07,0.0,0.000000e+00,1.708431e+08,0.000000,958900000.0,0.0000


In [28]:
experiments.head()

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,118,99.402021,1.0,0.744131,347.220398,1.0,0.716350,179.239940,1.0,0.510779,...,10,1,0,7,7,7,1,5,0,dikesnet
1,5,86.105155,1.5,0.600363,271.003716,1.5,0.393702,322.908620,1.5,0.808188,...,10,1,0,7,7,7,1,6,0,dikesnet
2,129,239.723086,1.5,0.255171,122.153781,1.5,0.224914,97.738293,10,0.687598,...,10,1,0,7,7,7,1,7,0,dikesnet
3,56,269.923744,10,0.946785,199.693619,10,0.995455,280.138357,1.5,0.402688,...,10,1,0,7,7,7,1,8,0,dikesnet
4,18,312.491930,1.5,0.863513,226.436587,1.5,0.804544,90.162261,10,0.174627,...,10,1,0,7,7,7,1,9,0,dikesnet


In [16]:
experiments_of_interest = experiments.loc[y]
outcomes_df = pd.DataFrame({k:v[y] for k,v in outcomes.items()})

#normalize outcomes on unit interval to ensure equal weighting of outcomes
x = outcomes_df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_outcomes = pd.DataFrame(x_scaled, columns=outcomes_df.columns)

In [17]:
n_scen = experiments.loc[y].shape[0]
indices = range(n_scen)
set_size = 5

n_scen
combinations = itertools.combinations(indices, set_size)
combinations = list(combinations)

In [18]:
#combinations
print(len(combinations))

575757


In [24]:
sampled_combinations = random.sample(combinations, 100)

In [32]:
distances = squareform(pdist(normalized_outcomes.values))

cores = os.cpu_count()
partial_function = functools.partial(find_maxdiverse_scenarios, distances)


#Difference ProcessPool en ThreadPool 
# https://superfastpython.com/threadpoolexecutor-vs-processpoolexecutor/#Comparison_of_ThreadPoolExecutor_vs_ProcessPoolExecutor
#with ProcessPoolExecutor(max_workers=cores) as executor:
with ThreadPoolExecutor(max_workers=cores) as executor:
    worker_data = np.array_split(combinations, cores)
    results = [e for e in executor.map(partial_function, worker_data)]
    results = list(itertools.chain.from_iterable(results))

In [33]:
results.sort(key=lambda entry:entry[0], reverse=True)
most_diverse = results[0]
most_diverse

([1.9859722743530344], array([ 1, 10, 13, 15, 34]))

In [36]:
selected = experiments.loc[most_diverse[1], ['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model']]
scenarios = [Scenario(f"{index}", **row) for index, row in selected.iterrows()]

In [39]:
scenarios

[Scenario({'A.0_ID flood wave shape': 5, 'A.1_Bmax': 86.10515505606122, 'A.1_Brate': 1.5, 'A.1_pfail': 0.600362982630473, 'A.2_Bmax': 271.00371575799124, 'A.2_Brate': 1.5, 'A.2_pfail': 0.39370221400841265, 'A.3_Bmax': 322.90861956267867, 'A.3_Brate': 1.5, 'A.3_pfail': 0.8081883631165219, 'A.4_Bmax': 241.06690016684723, 'A.4_Brate': 1.0, 'A.4_pfail': 0.4848215865929534, 'A.5_Bmax': 256.03963457360936, 'A.5_Brate': 1.5, 'A.5_pfail': 0.4807718874804777, 'discount rate 0': 2.5, 'discount rate 1': 4.5, 'discount rate 2': 2.5, '0_RfR 0': 0, '0_RfR 1': 0, '0_RfR 2': 1, '1_RfR 0': 1, '1_RfR 1': 0, '1_RfR 2': 1, '2_RfR 0': 1, '2_RfR 1': 0, '2_RfR 2': 1, '3_RfR 0': 0, '3_RfR 1': 1, '3_RfR 2': 0, '4_RfR 0': 0, '4_RfR 1': 0, '4_RfR 2': 1, 'A.1_DikeIncrease 0': 5, 'A.1_DikeIncrease 1': 2, 'A.1_DikeIncrease 2': 7, 'A.2_DikeIncrease 0': 6, 'A.2_DikeIncrease 1': 0, 'A.2_DikeIncrease 2': 1, 'A.3_DikeIncrease 0': 8, 'A.3_DikeIncrease 1': 5, 'A.3_DikeIncrease 2': 10, 'A.4_DikeIncrease 0': 10, 'A.4_DikeIn

# Mordm Run 

In [43]:
outcomes.keys()

Index(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs',
       'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage',
       'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths',
       'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs',
       'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage',
       'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
       'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs',
       'A.5_Expected Number of Deaths', 'RfR Total Costs',
       'Expected Evacuation Costs'],
      dtype='object')

In [42]:
levers = ['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model']

names_outcomes = ['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs',
       'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage',
       'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths',
       'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs',
       'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage',
       'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths',
       'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs',
       'A.5_Expected Number of Deaths', 'RfR Total Costs',
       'Expected Evacuation Costs']

In [ ]:
#Identify policies based on open exploration 

policies = [Policy('policy 1', **{'A.0_ID flood wave shape':, 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model'})]


#            Policy('policy 2', **{'4_RfR 0':1,
#                                   '4_RfR 1':1,
#                                   '4_RfR 2':1,
#                                   'A.5_DikeIncrease 0':5}),
#            Policy('policy 3', **{'1_RfR 0':1,
#                                   '2_RfR 1':1,
#                                   '3_RfR 2':1,
#                                   'A.3_DikeIncrease 0':5})]



In [ ]:
results = []
for scenario in scenarios:
    convergence_metrics = [HyperVolume(minimum=[0,0,0,0], maximum=[3, 2,1.01,1.01]),
                           EpsilonProgress()]
    epsilons = [0.1,]*len(model.outcomes)
    
    results.append(optimize(scenario, 1e4, model, convergence_metrics, epsilons))